In [1]:
import os
import sys

In [2]:
ROOT_DIR = "../../"
sys.path.insert(0, os.path.abspath(ROOT_DIR + "/src/scraping"))
from scraping import SteamScraper

In [3]:
import requests
import bs4
import pandas as pd
import numpy as np

In [4]:
sc = SteamScraper(ROOT_DIR)

In [5]:
sc.scrape_data()

In [6]:
sc.games

,name,href
0,Rust,https://store.steampowered.com/app/252490/Rust/
1,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...
2,Apex Legends™,https://store.steampowered.com/app/1172470/Ape...
3,Destiny 2,https://store.steampowered.com/app/1085660/Des...
4,Phasmophobia,https://store.steampowered.com/app/739630/Phas...
...,...,...
89963,PowerWash Simulator,https://store.steampowered.com/app/1290000/Pow...
89964,Hidden Shelter,https://store.steampowered.com/app/1422820/Hid...
89965,Realm Engine | Virtual Tabletop,https://store.steampowered.com/app/1427700/Rea...
89966,Go Gold Castle,https://store.steampowered.com/app/1443700/Go_...


***

In [11]:
%time sc.collect_sys_reqs(max_chunk=1)

Starting chunk 0:0-2000
Wall time: 15min 41s99


***

## Combining all chunks into one df

In [7]:
df = pd.read_csv(ROOT_DIR + "data/temp/chunksize_2000/game_sys_reqs_0.csv")
for i in range(1, 45):
    print("Combining df {}".format(i), end='\r')
    new_df = pd.read_csv(ROOT_DIR + "data/temp/chunksize_2000/game_sys_reqs_{}.csv".format(i))
    df = df.append(new_df)
df = df.reset_index(drop=True)

In [8]:
import datetime

In [9]:
sys_req_path = ROOT_DIR + "data/sys_reqs/"
if not os.path.exists(sys_req_path):
    os.mkdir(sys_req_path)
    
today = datetime.datetime.today()
date = f"{today.month}-{today.day}-{today.year}."
df.to_csv(sys_req_path + date + "sys_reqs.csv", index=False)